In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=631219b7aff2378b08d105c185e9b8b5e121407606fd06a509bf365a4fac1f8d
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
# installation des librairies
#! pip install pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegressionModel
from pyspark.ml.classification import RandomForestClassifier
from pyspark.sql.functions import col
import pandas as pd

In [ ]:
spark = SparkSession.builder.appName("Modele RF").getOrCreate()

In [ ]:
#df = spark.read.format("csv").option("path","/Users/mourtallangom/Downloads/diabetes.csv").load()
df = spark.read.csv('/content/diatebtes.csv', header = True, inferSchema=True) # predit le type de colonnes
#df = spark.read.format("csv").option("path", "/content/diabetes.csv").load()
df.show()

+----+----+----+----+----+----+-----+---+---------------+
|preg|plas|pres|skin|insu|mass| pedi|age|          class|
+----+----+----+----+----+----+-----+---+---------------+
|   6| 148|  72|  35|   0|33.6|0.627| 50|tested_positive|
|   1|  85|  66|  29|   0|26.6|0.351| 31|tested_negative|
|   8| 183|  64|   0|   0|23.3|0.672| 32|tested_positive|
|   1|  89|  66|  23|  94|28.1|0.167| 21|tested_negative|
|   0| 137|  40|  35| 168|43.1|2.288| 33|tested_positive|
|   5| 116|  74|   0|   0|25.6|0.201| 30|tested_negative|
|   3|  78|  50|  32|  88|31.0|0.248| 26|tested_positive|
|  10| 115|   0|   0|   0|35.3|0.134| 29|tested_negative|
|   2| 197|  70|  45| 543|30.5|0.158| 53|tested_positive|
|   8| 125|  96|   0|   0| 0.0|0.232| 54|tested_positive|
|   4| 110|  92|   0|   0|37.6|0.191| 30|tested_negative|
|  10| 168|  74|   0|   0|38.0|0.537| 34|tested_positive|
|  10| 139|  80|   0|   0|27.1|1.441| 57|tested_negative|
|   1| 189|  60|  23| 846|30.1|0.398| 59|tested_positive|
|   5| 166|  7

In [ ]:
#verifier le type de colonnes dans le df
df.printSchema()

root
 |-- preg: integer (nullable = true)
 |-- plas: integer (nullable = true)
 |-- pres: integer (nullable = true)
 |-- skin: integer (nullable = true)
 |-- insu: integer (nullable = true)
 |-- mass: double (nullable = true)
 |-- pedi: double (nullable = true)
 |-- age: integer (nullable = true)
 |-- class: string (nullable = true)



In [ ]:
# verifier les dimensions du df
print(df.count(), ':', len(df.columns))

768 : 9


In [ ]:
# calculer le nombre de personnes malades et personnes biens portantes dans le dataset
df.groupBy("class").count().show()

+---------------+-----+
|          class|count|
+---------------+-----+
|tested_positive|  268|
|tested_negative|  500|
+---------------+-----+



Nous avons 500 personnes malades sur l'ensemble des 768 personnes

In [ ]:
# Faire une etude descriptive dur le datset
df.describe().show()

+-------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+---------------+
|summary|              preg|             plas|              pres|              skin|              insu|              mass|              pedi|               age|          class|
+-------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+---------------+
|  count|               768|              768|               768|               768|               768|               768|               768|               768|            768|
|   mean|3.8450520833333335|     120.89453125|       69.10546875|20.536458333333332| 79.79947916666667|31.992578124999977|0.4718763020833327|33.240885416666664|           null|
| stddev|  3.36957806269887|31.97261819513622|19.355807170644777|15.952217567727642|115.24400235133803| 7.884160320

Le tableau montre que les statistiques basiques pour les colonnes numériques. La valeur minimale pour le glucose, l'insuline et la pression sanguine est égale à 0 !. Ces valeurs necessitent d'être nettoyées.

**Nottoyade des données : data cleaning**

In [ ]:
# Vérifier les valeurs manquantes dans le dataset
for col in df.columns:
  print(col + ":", df[df[col].isNull()].count())

preg: 0
plas: 0
pres: 0
skin: 0
insu: 0
mass: 0
pedi: 0
age: 0
class: 0


Notre dataset ne contient aucunes valeurs manquanes

In [ ]:
# Créer une fonction pour compter le nombre de valeur 0 et leur pourcentage par colonnes
def count_zeros(df, columns):
  for col in columns:
    num_zeros = df.filter(df[col] == 0).count()
    total_rows = df.count()
    percentage = (num_zeros / total_rows) * 100
    print("{} : {} ({:.2f}%)".format(col, num_zeros, percentage))


In [ ]:
liste_cols = ['plas', 'pres', 'skin', 'insu', 'mass']
count_zeros(df, liste_cols)

plas : 5 (0.65%)
pres : 35 (4.56%)
skin : 227 (29.56%)
insu : 374 (48.70%)
mass : 11 (1.43%)


- Ces pourcentages indiquent que la colonne Glucose a le plus faible pourcentage de valeurs nulles, avec seulement 0,65%.
- La colonne Insulin a le pourcentage le plus élevé de valeurs nulles, avec 48.70%.
- Les autres colonnes ont des pourcentages de valeurs nulles compris entre 1,43% et 29,70%.

In [ ]:
# Afficher la valeur moyenne pour chaque colonne  et faire le remplacement
for i in df.columns[1:6]:
  mean_val = df.agg({i:'mean'}).first()[0]
  print("la valeur moyenne de la colonne {} est : {}".format(i, int(mean_val)))
  # update the values : si la condition (val ==0) est vrai
  df = df.withColumn(i, when(df[i]==0, int(mean_val)).otherwise(df[i]))

df.show(15)

la valeur moyenne de la colonne plas est : 120
la valeur moyenne de la colonne pres est : 69
la valeur moyenne de la colonne skin est : 20
la valeur moyenne de la colonne insu est : 79
la valeur moyenne de la colonne mass est : 31
+----+----+----+----+----+----+-----+---+---------------+
|preg|plas|pres|skin|insu|mass| pedi|age|          class|
+----+----+----+----+----+----+-----+---+---------------+
|   6| 148|  72|  35|  79|33.6|0.627| 50|tested_positive|
|   1|  85|  66|  29|  79|26.6|0.351| 31|tested_negative|
|   8| 183|  64|  20|  79|23.3|0.672| 32|tested_positive|
|   1|  89|  66|  23|  94|28.1|0.167| 21|tested_negative|
|   0| 137|  40|  35| 168|43.1|2.288| 33|tested_positive|
|   5| 116|  74|  20|  79|25.6|0.201| 30|tested_negative|
|   3|  78|  50|  32|  88|31.0|0.248| 26|tested_positive|
|  10| 115|  69|  20|  79|35.3|0.134| 29|tested_negative|
|   2| 197|  70|  45| 543|30.5|0.158| 53|tested_positive|
|   8| 125|  96|  20|  79|31.0|0.232| 54|tested_positive|
|   4| 110|  92

les valeurs sont bien remlacées

Construire et entrainer le modèle de machine learning:


Feature ingeneering

In [ ]:
# Calculer le corrélation entre le valeur de réponse et les autres variables
#for col in df.columns:
  #print('la correlation de {} avec la variable class est {}.'.format(col, df.stat.corr('class', col)))
  #print(f'La correlation de  {col} avec la variable class est {df.stat.corr('class', col)}.')

X=pd.DataFrame(np.c_[df['preg'],df['plas'],df['pres'],df['insu'],df['mass'],df['pedi'],df['age']], columns = ['preg','plas','pres','skin','insu','mass','pedi','age'])
Y = df['class']

In [ ]:
# creer un vectorAssembler : c'est un feature transformer qui merge les différentes colonnes dans un seul vecteur (features).
inputCols = ['preg' , 'plas' , 'pres', 'skin' , 'insu' , 'mass' , 'pedi' ,'age'  ]
assembler = VectorAssembler(inputCols= inputCols, outputCol='features')
output_data = assembler.transform(df)

In [ ]:
# verifier si la colonne est rajoutée au dataframe
output_data.printSchema()

root
 |-- preg: integer (nullable = true)
 |-- plas: integer (nullable = true)
 |-- pres: integer (nullable = true)
 |-- skin: integer (nullable = true)
 |-- insu: integer (nullable = true)
 |-- mass: double (nullable = true)
 |-- pedi: double (nullable = true)
 |-- age: integer (nullable = true)
 |-- class: string (nullable = true)
 |-- features: vector (nullable = true)



Entrainer le modèle

In [ ]:
# séléctionner les colonnes d'interer
final_df = output_data.select('features', 'class')
final_df.show(2)

+--------------------+---------------+
|            features|          class|
+--------------------+---------------+
|[6.0,148.0,72.0,3...|tested_positive|
|[1.0,85.0,66.0,29...|tested_negative|
+--------------------+---------------+
only showing top 2 rows



In [ ]:
# Diviser les données en ensembles d'entraînement(70%) et de test(30%)
train, test = df.randomSplit([0.7, 0.3], seed=42)

In [ ]:
# créer le modèle
models = LogisticRegression(labelCol='class')

In [ ]:
# Créer le modèle
rf = RandomForestClassifier(numTrees=10, maxDepth=5, seed=42)

In [ ]:
# entrainer le modèle
model = models.fit(df)

In [ ]:
# afficher le sommaire
summary = model.summary

In [ ]:
## les prédictions
summary.predictions.show()
summary.predictions.describe().show()